# Implementing an LLM-powered recommendation system

## Data Preprocessing

In [1]:
import pandas as pd
import tiktoken
import lancedb
from openai import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.callbacks import get_openai_callback
from langchain_community.vectorstores import LanceDB
import streamlit as st

openai_api_key = ""
client = OpenAI(api_key=openai_api_key)

anime = pd. read_csv('data/anime_with_synopsis.csv')
anime.head()

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...


In [2]:
anime = anime.dropna()

In [3]:

anime['combined_info'] = anime.apply(lambda row: f"Title: {row['Name']}. Overview: {row['sypnopsis']} Genres: {row['Genres']}", axis=1)
anime['combined_info'][0]



'Title: Cowboy Bebop. Overview: In the year 2071, humanity has colonized several of the planets and moons of the solar system leaving the now uninhabitable surface of planet Earth behind. The Inter Solar System Police attempts to keep peace in the galaxy, aided in part by outlaw bounty hunters, referred to as "Cowboys." The ragtag team aboard the spaceship Bebop are two such individuals. Mellow and carefree Spike Spiegel is balanced by his boisterous, pragmatic partner Jet Black as the pair makes a living chasing bounties and collecting rewards. Thrown off course by the addition of new members that they meet in their travels—Ein, a genetically engineered, highly intelligent Welsh Corgi; femme fatale Faye Valentine, an enigmatic trickster with memory loss; and the strange computer whiz kid Edward Wong—the crew embarks on thrilling adventures that unravel each member\'s dark and mysterious past little by little. Well-balanced with high density action and light-hearted comedy, Cowboy Bebo

## Embeddings

In [4]:
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

encoding = tiktoken.get_encoding(embedding_encoding)

# omit descriptions that are too long to embed
anime["n_tokens"] = anime.combined_info.apply(lambda x: len(encoding.encode(x)))
anime = anime[anime.n_tokens <= max_tokens]
len(anime)

269

In [5]:
anime.head()

,MAL_ID,Name,Score,Genres,sypnopsis,combined_info,n_tokens
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever...",Title: Cowboy Bebop. Overview: In the year 207...,245
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ...",Title: Cowboy Bebop: Tengoku no Tobira. Overvi...,199
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0...",Title: Trigun. Overview: Vash the Stampede is ...,252
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...,Title: Witch Hunter Robin. Overview: ches are ...,125
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...,Title: Bouken Ou Beet. Overview: It is the dar...,188


In [6]:
def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding


anime["embedding"] = anime.combined_info.apply(lambda x: get_embedding(x, model=embedding_model))
anime.head()

,MAL_ID,Name,Score,Genres,sypnopsis,combined_info,n_tokens,embedding
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever...",Title: Cowboy Bebop. Overview: In the year 207...,245,"[0.01118044275790453, -0.0417056530714035, -0...."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ...",Title: Cowboy Bebop: Tengoku no Tobira. Overvi...,199,"[0.0051825987175107, -0.0393059179186821, -0.0..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0...",Title: Trigun. Overview: Vash the Stampede is ...,252,"[-0.04254202917218208, -0.027669398114085197, ..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...,Title: Witch Hunter Robin. Overview: ches are ...,125,"[0.011605343781411648, -0.03196525201201439, -..."
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...,Title: Bouken Ou Beet. Overview: It is the dar...,188,"[-0.01851538196206093, -0.048899248242378235, ..."


In [7]:
anime.rename(columns = {'embedding': 'vector'}, inplace = True)
anime.rename(columns = {'combined_info': 'text'}, inplace = True)
anime.to_pickle('data/anime.pkl')

## Start working with LLMs

In [8]:
uri = "dataset/sample-anime-lancedb"

db = lancedb.connect(uri)
table = db.open_table("anime")

# embeddings = OpenAIEmbeddings(engine="text-embedding-ada-002")
embeddings = OpenAIEmbeddings(
    deployment="SL-document_embedder",
    model="text-embedding-ada-002",
    show_progress_bar=True,
    openai_api_key=openai_api_key)

docsearch = LanceDB(connection=table, embedding=embeddings)

# simple similarity computation
# query = "I'm looking for an animated action movie. What could you suggest to me?"
# docs = docsearch.similarity_search(query, k=1)

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo-1106",
    temperature=0,
    api_key=openai_api_key
)

## Prompt engineering

In [9]:
template = """You are a movie recommender system that help users to find anime that match their preferences. 
Use the following pieces of context to answer the question at the end. 
For each question, suggest three anime, with a short description of the plot and the reason why the user migth like it.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Your response:"""

PROMPT = PromptTemplate(
    template=template, input_variables=["context", "question"])

chain_type_kwargs = {"prompt": PROMPT}

qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type="stuff",
                                       retriever=docsearch.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs=chain_type_kwargs)

# Streamlit app
st.title('Anime Recommendation System')

query = st.text_input('Enter your query:', '')

if query:
    with st.spinner('Finding recommendations...'):
        with get_openai_callback() as cb:
            result = qa_chain({"query": query})
        
        recommendations = result['result']
        st.subheader('Top 3 Recommendations:')
        for i, anime in enumerate(recommendations):
            st.write(f"**Recommendation {i+1}:**")



c:\Users\ravik\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


  0%|          | 0/1 [00:00<?, ?it/s]

1. Kimi ga Nozomu Eien - This anime is a romantic drama that explores the complexities of relationships and the impact of tragedy on love. The emotional depth and character development make it a compelling watch for fans of romance.

2. Rurouni Kenshin: Meiji Kenkaku Romantan - Tsuioku-hen - While primarily an action and historical anime, this series also delves into the romantic aspects of the protagonist's life, offering a unique blend of romance and drama within a samurai setting.

3. Rurouni Kenshin: Meiji Kenkaku Romantan - Ishinshishi e no Chinkonka - This anime features a romantic subplot amidst its historical and samurai themes, providing a compelling and emotional storyline for fans of romance.
